<a href="https://colab.research.google.com/github/halfbug/colab/blob/main/retriever-vector-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain and OpenAI installation**


In [1]:
try:
  import openai
  from langchain_openai import ChatOpenAI
except:
    !pip install openai
    !pip install langchain
    !pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [2]:
import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.chat_models import ChatOpenAI
import datetime
current_date = datetime.datetime.now().date()
# llm_name = "gpt-4"
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.invoke("Hello world!")

gpt-3.5-turbo


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

# Scraper

In [4]:
import re
import requests
from bs4 import BeautifulSoup

def extract_availability(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            #  availability_pattern = re.compile(r"(?:stock|stock|availability|Number\sof\sItems\s*(\d+)|in\s*stock|(\w+)\s*stock\s*(\w+))", re.IGNORECASE)

            availability_pattern = re.compile(r"(?:stock|stock|availability|in\s*stock)", re.IGNORECASE)
            match = re.search(availability_pattern, text_content)
            # print(match.groups())
            if match:
                # print(match.group(0))
                return f"Availability: {match.group(0).strip()}"
        # If specific keywords not found, return "Scraped Availability: Unknown"
        return "Availability: Unknown"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "Availability: Unknown"

In [5]:
def scrape_text(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

         # Step 3: Extract metadata
        meta_tags = soup.find_all('meta')
        metadata = {}
        for tag in meta_tags:
            if 'name' in tag.attrs:
                name = tag.attrs['name']
                content = tag.attrs.get('content', '')
                metadata[name] = content
            elif 'property' in tag.attrs:  # For OpenGraph metadata
                property = tag.attrs['property']
                content = tag.attrs.get('content', '')
                metadata[property] = content

        # Display the metadata
        # for key, value in metadata.items():
        #     print(f"{key}: {value}")

        # print( metadata)

        meta_tag = extract_product_info([metadata])

            # Find all link elements with the 'as' attribute set to 'image'
        image_links = soup.find_all('link', {'rel': 'preload', 'as': 'image'})

        # Extract and print the href attributes
        images = ''
        for link in image_links:
            image_url = link.get('href')
            if image_url:
                if image_url.startswith("//"):
                  # Add protocol (http assumed):
                  image_url= f"http:{image_url}"
                images = images +","+ image_url
                # print(image_url)

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            return text_content + meta_tag + "Images : " + images
        return f"Content: Unknown on {url}"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return f"Content: Unknown on {url}"

In [6]:
def google_scraper(query) :
    try:
        response = requests.get("https://www.google.com/search?q="+query.lower().replace(' ', '+'))
        soup = BeautifulSoup(response.content, 'html.parser')

        # find first five link's content
        anchor_tags = soup.find_all('a', href=True)
        hrefs = []
        content = ""
        for tag in anchor_tags:
          href = tag.get('href')
          if href and href.startswith("/url?q=https") and "google" not in href.lower():  # Check if href exists (it might be None)
            parts = href.split('&', 1)
            parts = parts[0].split('/url?q=',1)
            # print(parts)
            hrefs.append(parts[1])
            scraped_content = scrape_text(parts[1])
            content += scraped_content + "\n ***"
            # print(scraped_content)
        # print(hrefs[:5])



        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # print("\033[34m" + text_content )
            return text_content + content
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "No search result found: Unknown"
google_scraper("Apotheke 3 wick candles collection")

Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined
Error occurred while scraping: name 'extract_product_info' is not defined


'Google×Please clickhereif you are not redirected within a few seconds.AllImagesShoppingNewsMapsVideosBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatim3-Wick Candles | APOTHEKEapothekeco.com › collections › 3-wick-candlesMade with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Charcoal 3-Wick Candle - Apothekeapothekeco.com › products › charcoal-3-wick-candleRating4.4(125)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a chic matte-black glass vessel. The FragranceCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a luxe white-translucent glass vessel. The\xa0...APOTHEKE M

In [7]:
import re

def extract_product_info(meta_tags):
    product_info = ""
    for meta_tag in meta_tags:
        if 'og:title' in meta_tag:
            product_info += f"\n ***Title: {meta_tag['og:title']}\n"
        if 'og:description' in meta_tag:
            product_info += f"Description: {meta_tag['og:description']}\n"
        if 'og:price:amount' in meta_tag and 'og:price:currency' in meta_tag:
            product_info += f"Price: {meta_tag['og:price:amount']} {meta_tag['og:price:currency']}\n"
        if 'og:image' in meta_tag:
                if len(meta_tag['og:image']) <= 200:  # Check if URL length is less than or equal to 200
                    product_info += f"Image URL: {meta_tag['og:image']} \n"
        if 'og:url' in meta_tag:
            product_info += f"URL: {meta_tag['og:url']} \n"
            # Call function to extract availability information
            availability_info = extract_availability(meta_tag['og:url'])
            product_info += f"{availability_info}\n"
            # Extract ASIN from URL if it's an Amazon link
            asin_match = re.search(r'/dp/(\w+)', meta_tag['og:url'])
            if asin_match:
                product_info += f"ASIN: {asin_match.group(1)} \n"
        if 'og:site_name' in meta_tag:
            product_info += f"Site Name: {meta_tag['og:site_name']} \n"
        if 'og:availability' in meta_tag:
            product_info += f"Availability: {meta_tag['og:availability']} \n"
        if 'og:type' in meta_tag:
                product_info += f"type: {meta_tag['og:type']} \n"
        else:
          product_info += f"type: text \n"
    return product_info

# Example usage:
# meta_tags = [{'apple-itunes-app': 'app-id=529356019, app-argument=https://www.crateandbarrel.com/apotheke-canvas-scented-3-wick-candle/s569504',
#  'og:image': 'https://cb.scene7.com/is/image/Crate/ApothekeCnvs3WickCandleSSF23', 'og:type': 'product', 'og:image:alt': 'Apotheke Canvas-Scented 3-Wick Candle',
#   'twitter:card': 'summary_large_image', 'twitter:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:site_name': 'Crate & Barrel', 'og:availability':
#    'InStock', 'og:price:amount': '$78.00', 'author': 'Crate & Barrel', 'og:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:price:currency': 'USD', 'language':
#     'English', 'og:description': 'Shop Apotheke Canvas-Scented 3-Wick Candle.  This hand-poured candle fills the room with clean, refreshing scent.  Infused with essences of crisp linen, white musk,
#      lily of the valley and dew drop, the soy-blend wax candle burns for many hours releasing a sweet, fresh fragrance.', 'og:price:standard_amount': '$78.00', 'twitter:site': '@CrateandBarrel',
#      'viewport': 'width=device-width, initial-scale=1.0', 'fb:admins': '1769841725', 'og:url': 'https://www.amazon.com/dp/B09RTPD6ZN/ref=tsm_1_fb_lk'}]
# product_info = extract_product_info(meta_tags)
# print(product_info)

# Google Search API

In [8]:
from googleapiclient.discovery import build

API_KEY = userdata.get('GOOGLE_SEARCH_API')
# os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')

service = build("customsearch", "v1", developerKey=API_KEY)
# clientProd = build("shopping", "v1", developerKey=API_KEY)

In [9]:
def search_google(query):
    """
    Performs a custom search  using the Google Custom Search API.

    Args:
        query (str): The search query to use.
        num_results (int, optional): The number of image search results to return (default: 3).

    Returns:
        str: A string containing information about the image search results. Each result includes the image URL and its corresponding link.
    """
    # user_query= extract_text_between_hashes(query)
    results = service.cse().list(
        q=query, cx=userdata.get('GOOGLE_CSE_ID'),  # Replace with your custom search engine ID
        # searchType="image",  # Specify image search
        num=3
    ).execute()

    search_results = []
    for item in results.get("items", []):
        for key, value in item.items():
          print("\033[32m" + f"{key}: {value}" + "\033[0m")
        image_url = item["pagemap"]["cse_image"][0]["src"]
        print(item["pagemap"]["metatags"])
        search_results.append({"link":item["link"] ,
                              "title": item["title"] ,
                              "snippets":item["snippet"],
                              "image": image_url,
                              "meta": extract_product_info(item["pagemap"]["metatags"])})
        # search_results += extract_product_info(item["pagemap"]["metatags"])
        # search_results += f"image2: {image_url} \n"
        # search_results += f"link: {item['link']}"

    len(search_results)
    return search_results

# Example usage:
image_results = search_google("Apotheke 3 wick candles collection price images")
# image_results
print(image_results)

kind: customsearch#result
title: Apotheke 3-Wick Candles Collection : Health ... - Amazon.com
htmlTitle: Apotheke 3-Wick Candles Collection : Health ... - Amazon.com
link: https://www.amazon.com/Scented-Candles-Home-Soy-Wax-Jar-Candle/dp/B07PMY9PK6
displayLink: www.amazon.com
snippet: Captivating Fragrance: Transform any living space with the unforgettable scent of Charcoal 3-Wick Candle from APOTHEKE. Our candles are hand poured and made ...
htmlSnippet: Captivating Fragrance: Transform any living space with the unforgettable scent of Charcoal <b>3</b>-<b>Wick Candle</b> from <b>APOTHEKE</b>. Our <b>candles</b> are hand poured and made&nbsp;...
formattedUrl: https://www.amazon.com/Scented-Candles-Home-Soy.../B07PMY9PK6
htmlFormattedUrl: https://www.amazon.com/Scented-<b>Candles</b>-Home-Soy.../B07PMY9PK6
pagemap: {'cse_thumbnail': [{'src': 'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSn_XK9lxk8YHeoZ-1cwFl6vAw52HIMW2FN2RbGuGT2KqCwycqvSOYtY-o', 'width': '238', 'height': '212'}

In [10]:
def combined_scraped_api_search(query):
  scraped_results = google_scraper(query)
  search_api_results = search_google(query)
  return f"""{scraped_results} \n {search_api_results} """


# RAG

In [11]:
column =  "Apotheke 3 wick candles collection"

In [12]:
google_scraper(column)

'Google×Please clickhereif you are not redirected within a few seconds.AllImagesShoppingNewsMapsVideosBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatim3-Wick Candles | APOTHEKEapothekeco.com › collections › 3-wick-candlesMade with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Charcoal 3-Wick Candle - Apothekeapothekeco.com › products › charcoal-3-wick-candleRating4.4(125)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a chic matte-black glass vessel. The FragranceCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a luxe white-translucent glass vessel. The\xa0...Apotheke 3

## Document Splitting

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 150)


In [14]:
splits = text_splitter.split_text(google_scraper(column))
len(splits)

209

In [15]:
splits


['Google×Please clickhereif you are not redirected within a few seconds.AllImagesShoppingNewsMapsVideosBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatim3-Wick Candles | APOTHEKEapothekeco.com › collections › 3-wick-candlesMade with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Charcoal 3-Wick Candle - Apothekeapothekeco.com ›',
 'fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Charcoal 3-Wick Candle - Apothekeapothekeco.com › products › charcoal-3-wick-candleRating4.4(125)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a chic matte-black glass vessel. The FragranceCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Ca

## Embeddings and Vectorstores
Let's take our splits and embed them.

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [17]:
!pip install chromadb

from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any
import sys
sys.path.append('../..')




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 8.

In [18]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [19]:
vectordb = Chroma.from_texts(splits, embedding=embedding)

In [20]:
print(vectordb._collection.count())

209


In [21]:
vectordb.similarity_search(column+" upc description", k=5)

[Document(page_content='URL: https://apothekeco.com/collections/3-wick-candles \nAvailability: Unknown\nSite Name: APOTHEKE \ntype: product.group \nImages : ,http://apothekeco.com/cdn/shop/collections/3-wick-candles-681433.jpg?v=1661870647&width=1950'),
 Document(page_content='***Title: 3-Wick Candles | APOTHEKE\nDescription: Made with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning and exceptional scent experience.\nImage URL: http:collections/3-wick-candles-681433.jpg \nURL: https://apothekeco.com/collections/3-wick-candles \nAvailability: Unknown\nSite Name: APOTHEKE \ntype: product.group'),
 Document(page_content='3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a glossy blue-translucent glass vessel. The\xa0...Apotheke 3-Wick Candles Collection - Amazon.comwww.amazon.com › Scented-Candles-Home-Soy-Wax-Jar-CandleRating4.2(147)

In [22]:
vectordb.max_marginal_relevance_search(column+" upc",k=5)

[Document(page_content='URL: https://apothekeco.com/collections/3-wick-candles \nAvailability: Unknown\nSite Name: APOTHEKE \ntype: product.group \nImages : ,http://apothekeco.com/cdn/shop/collections/3-wick-candles-681433.jpg?v=1661870647&width=1950'),
 Document(page_content='The FragranceCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a luxe white-translucent glass vessel. The\xa0...APOTHEKE Market Collection Luxury Scented 3-Wick Jar Candle ...www.amazon.com › APOTHEKE-Market-Collection-Luxury-ScentedRating4.2(147)·$72.95Buy APOTHEKE Market Collection Luxury Scented 3-Wick Jar Candle, Meyer Lemon'),
 Document(page_content='***Title: Apotheke - Tonka Oak 3 Wick Candle\nDescription: Apotheke Tonka Oak 3 Wick Candle fills any room in your home with hours of unforgettable bliss. Made of perfume grade fragrance oils, 

In [36]:
retriever = vectordb.as_retriever()

In [37]:
retriever.get_relevant_documents("where did harrison work?")


[Document(page_content='RewardsBecome an AmbassadorBlogFAQPrivacyTerms & ConditionsMSRP/MAP PolicyApply for WholesaleContact UsSign up to receive exclusive discounts, stay up to date with the latest releases, and learn a little bit about... us!Right arrow longCurrencyUnited States (USD\xa0$)Australia(USD\xa0$)Austria(USD\xa0$)Belgium(USD\xa0$)Bulgaria(USD\xa0$)Canada(USD\xa0$)Czechia(USD\xa0$)Denmark(USD\xa0$)Estonia(USD\xa0$)Finland(USD\xa0$)France(USD\xa0$)Germany(USD\xa0$)Hong Kong'),
 Document(page_content='RewardsBecome an AmbassadorBlogFAQPrivacyTerms & ConditionsMSRP/MAP PolicyApply for WholesaleContact UsSign up to receive exclusive discounts, stay up to date with the latest releases, and learn a little bit about... us!Right arrow longCurrencyUnited States (USD\xa0$)Australia(USD\xa0$)Austria(USD\xa0$)Belgium(USD\xa0$)Bulgaria(USD\xa0$)Canada(USD\xa0$)Czechia(USD\xa0$)Denmark(USD\xa0$)Estonia(USD\xa0$)Finland(USD\xa0$)France(USD\xa0$)Germany(USD\xa0$)Hong Kong'),
 Document(page

# LangChain Expression Language (LCEL)

In [25]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableMap

In [26]:
template = """
As a search analyst, your task is to find specific information online. We've provided context to assist:

{context}

Answer the following question based solely on the context:

Question: {question}

Provide the requested information in this format:

Brand name:
Product name:
Product image(s): [provide at least one link]
Color: [list colors, separated by commas]
Category: [identify the product's category on Amazon]
Description: [include a brief product description]
Sold on Amazon: true / false
Amazon price: [indicate the price range]
ASIN: [list ASIN codes, separated by commas]
UPC: [mention the UPC]

If the information isn't available, conduct your own search and provide an answer.
"""
prompt = ChatPromptTemplate.from_template(template)

In [27]:
model = llm
output_parser = StrOutputParser()



In [43]:
chain = RunnableMap({
    "context": lambda x:retriever.get_relevant_documents(x["question"]),       #google_scraper(x["question"]),     #combined_scraped_api_search(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [44]:
print(chain.invoke({"question": "Apotheke 3-Wick Candles Collection charcoal"}))

Brand name: Apotheke
Product name: Charcoal 3-Wick Candle
Product image(s): [http://apothekeco.com/cdn/shop/products/AP03-CH_Charcoal_26oz_3WickCandle_1000x1000.jpg?v=1704389442]
Color: Matte black
Category: Home & Kitchen
Description: Our number 1 best seller, Charcoal, featured in our signature matte black vessel will elevate any room in your home. Notes of cedarwood and sandalwood blended with smokey amber and oud come together to create a broody full-bodied fragrance.
Sold on Amazon: true
Amazon price: $78.00
ASIN: B08K3ZQZ3P
UPC: 850000236013


In [45]:
from langchain.callbacks import get_openai_callback


In [47]:
with get_openai_callback() as cb:
    result = chain.invoke({"question": "Apotheke 3-Wick Candles Collection charcoal"})
    print(cb)

Tokens Used: 836
	Prompt Tokens: 676
	Completion Tokens: 160
Successful Requests: 1
Total Cost (USD): $0.0013340000000000001
